In [1]:
# load packages

import os
import numpy as np
import pandas as pd
from qsprpred.data import QSPRDataset
from sklearn.impute import SimpleImputer
from qsprpred.data import QSPRDataset, RandomSplit
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.data.descriptors.sets import SmilesDesc
from qsprpred.data.sampling.splits import GBMTRandomSplit, GBMTDataSplit, ManualSplit
from split import random_global_equilibrated_random_split, dissimilaritydrive_global_balanced_cluster_split
from qsprpred.extra.gpu.models.chemprop import ChempropModel
from qsprpred.models import CrossValAssessor, TestSetAssessor
from qsprpred.models.early_stopping import EarlyStoppingMode
# from qsprpred.plotting.regression import CorrelationPlot
from plotting_regression_MT import CorrelationPlot

Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [2]:
df=pd.read_csv('/home/s2861704/ppk/test_kin/output/TK/TK_split.csv', sep=',')
display(df)
print(df.columns)
print(type(df.loc[df.index[0], 'P00533']))

,SMILES,O60674,P00519,P00533,P04626,P04629,P06213,P06239,P07333,P07949,...,Q06187,Q06418,Q07912,Q08345,Q08881,Q12866,Q16288,Q16620,Q16832,split
0,BrC(Cn1ncc2c(N3CCCC3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
1,BrC(Cn1ncc2c(N3CCCCC3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
2,BrC(Cn1ncc2c(NCc3ccccc3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
3,Brc1cc(CNc2ncc(Br)c(Nc3cc(C4CC4)[nH]n3)n2)on1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
4,Brc1cc2c(NCc3ccccc3)ncnc2s1,NaN,NaN,6.62,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36999,c1nc2c(Nc3ccc(N4CCOCC4)cc3)nc(-c3ccc4c[nH]nc4c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37000,c1nc2c(Nc3ccc(N4CCOCC4)cc3)nc(-c3ccc4c[nH]nc4c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37001,c1ncc(-c2cc(CNc3nc(Nc4cc(C5CC5)[nH]n4)cc(N4CCO...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37002,c1ncc(CN2CCC(n3cnc4cnc5[nH]ccc5c43)CC2)cn1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train


Index(['SMILES', 'O60674', 'P00519', 'P00533', 'P04626', 'P04629', 'P06213',
       'P06239', 'P07333', 'P07949', 'P08069', 'P08581', 'P09581', 'P09619',
       'P10721', 'P11362', 'P12931', 'P16234', 'P17948', 'P21802', 'P22455',
       'P22607', 'P23458', 'P29317', 'P29597', 'P30530', 'P35968', 'P36888',
       'P43405', 'P52333', 'P54760', 'Q02763', 'Q05397', 'Q06187', 'Q06418',
       'Q07912', 'Q08345', 'Q08881', 'Q12866', 'Q16288', 'Q16620', 'Q16832',
       'split'],
      dtype='object')
<class 'numpy.float64'>


In [3]:
# print(len(df.columns))
# max_Nans = len(df.columns) - 5
# print(max_Nans)

In [4]:
targets_kinase200 = df.columns[1:42].tolist()
display(targets_kinase200)

['O60674',
 'P00519',
 'P00533',
 'P04626',
 'P04629',
 'P06213',
 'P06239',
 'P07333',
 'P07949',
 'P08069',
 'P08581',
 'P09581',
 'P09619',
 'P10721',
 'P11362',
 'P12931',
 'P16234',
 'P17948',
 'P21802',
 'P22455',
 'P22607',
 'P23458',
 'P29317',
 'P29597',
 'P30530',
 'P35968',
 'P36888',
 'P43405',
 'P52333',
 'P54760',
 'Q02763',
 'Q05397',
 'Q06187',
 'Q06418',
 'Q07912',
 'Q08345',
 'Q08881',
 'Q12866',
 'Q16288',
 'Q16620',
 'Q16832']

In [5]:
for target in targets_kinase200:
    df.loc[df[target] < 10.5, target] = np.nan
display(df)

,SMILES,O60674,P00519,P00533,P04626,P04629,P06213,P06239,P07333,P07949,...,Q06187,Q06418,Q07912,Q08345,Q08881,Q12866,Q16288,Q16620,Q16832,split
0,BrC(Cn1ncc2c(N3CCCC3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
1,BrC(Cn1ncc2c(N3CCCCC3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
2,BrC(Cn1ncc2c(NCc3ccccc3)ncnc21)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
3,Brc1cc(CNc2ncc(Br)c(Nc3cc(C4CC4)[nH]n3)n2)on1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
4,Brc1cc2c(NCc3ccccc3)ncnc2s1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36999,c1nc2c(Nc3ccc(N4CCOCC4)cc3)nc(-c3ccc4c[nH]nc4c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37000,c1nc2c(Nc3ccc(N4CCOCC4)cc3)nc(-c3ccc4c[nH]nc4c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37001,c1ncc(-c2cc(CNc3nc(Nc4cc(C5CC5)[nH]n4)cc(N4CCO...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
37002,c1ncc(CN2CCC(n3cnc4cnc5[nH]ccc5c43)CC2)cn1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train


In [8]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
df_pol = df.dropna(axis=0, thresh=4)
display(df_pol)

,SMILES,O60674,P00519,P00533,P04626,P04629,P06213,P06239,P07333,P07949,P08069,P08581,P09581,P09619,P10721,P11362,P12931,P16234,P17948,P21802,P22455,P22607,P23458,P29317,P29597,P30530,P35968,P36888,P43405,P52333,P54760,Q02763,Q05397,Q06187,Q06418,Q07912,Q08345,Q08881,Q12866,Q16288,Q16620,Q16832,split
3922,CC(=O)N1CCCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1,10.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
23277,CS(=O)(=O)N1CCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1,10.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train


In [ ]:
# CUTOFF = pChEMBL 10.5
# 2+ (2)
#   1. CC(=O)N1CCCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [3922]
#   2. CS(=O)(=O)N1CCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [23277]
# 1+ (72)

In [ ]:
# CUTOFF = pChEMBL 10.0 (all except 7487 include target O60674, all except 7487, 23559, 34170 include P52333 )
# 2+ (20)
#   1. CC(=O)N1CCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [3885]
#   2. CC(=O)N1CCCC(Nc2nc(C)ccc2-c2cnc3[nH]ccc3n2)C1 [3921]
#   3. CC(=O)N1CCCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [3922]
#   4. CC(=O)N1CCCCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [3929]
#   5. CC(C)CS(=O)(=O)N1CC(C)CC(Nc2nc(N3CCN(C)CC3)ncc2-c2cnc3[nH]ccc3n2)C1 [5133]
#   6. CC(C)CS(=O)(=O)N1CCCC(Nc2nc(N3CCOCC3)ncc2-c2cnc3[nH]ccc3n2)C1 [5142]
#   7. CC(C)CS(=O)(=O)N1CCCC(Nc2nccnc2-c2cnc3[nH]ccc3n2)C1 [5149]
#   8. CC1CNC(=O)c2cnn3ccc(nc23)NC(C)c2cc(F)ccc2O1 [7487]
#   9. CN(C)C1CCN(c2ncc(-c3cnc4[nH]ccc4n3)c(NC3CCCN(S(C)(=O)=O)C3)n2)C1 [11302]
#   10. COC(=O)N1CCCCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [14850]
#   11. COCCNc1ncc(-c2cnc3[nH]ccc3n2)c(NC2CCCN(S(C)(=O)=O)C2)n1 [15647]
#   12. COc1ncc(-c2cnc3[nH]ccc3n2)c(NC2CCCN(S(C)(=O)=O)C2)n1 [23079]
#   13. CS(=O)(=O)N1CCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [23277]
#   14. CS(=O)(=O)N1CCCC(Nc2nc(N3CC(C#N)C3)ncc2-c2cnc3[nH]ccc3n2)C1 [23294]
#   15. CS(=O)(=O)N1CCCC(Nc2ncccc2-c2cnc3[nH]cc(C4CC4)c3n2)C1 [23306]
#   16. CS(=O)(=O)N1CCCCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1 [23322]
#   17. CSc1ccc(OC(F)F)c(-c2[nH]ncc2NC(=O)c2cnc3cccnn23)c1 [23559]
#   18. Cc1c[nH]c2ncc(-c3cccnc3NC3CCCN(S(C)(=O)=O)C3)nc12 [24157]
#   19. Cc1ccc(-c2cnc3[nH]ccc3n2)c(NC2CCCN(S(C)(=O)=O)C2)n1	[25444]
#   20. O=C(Nc1c[nH]nc1-c1cc2c[nH]nc2cc1OC(F)F)c1cnn2cccnc12 [34170]
# 1+ (257)

In [ ]:
# CUTOFF = pChEMBL 9.5
# 3+ (1)
#   1. COc1cc(OC)c(F)c(N2Cc3cnc4[nH]ccc4c3N(C)C2=O)c1F [18138]
# 2+ (76)
# 1+ (740)

In [ ]:
# CUTOFF = pChEMBL 9.0
# 4+ (1)
#   1. OCC(Nc1cnc(-c2cc(Cl)ccc2O)c(-c2ccc3cnccc3c2)c1)c1ccccc1 [36186]
# 3+ (16)
#   2. CC(C)n1cc(C(=O)Nc2ccc(Oc3ccnc(NC(=O)C4CC4)c3)c(F)c2)c(=O)n(-c2ccc(F)cc2)c1=O [5725]
#   3. CCc1cc(O)c(F)cc1-c1cc(NC(C)CO)c2nc(NCCCCCN(C)C)ncc2c1 [10501]
#   4. CCc1cc(O)c(F)cc1-c1cc(NC2CCN(C(C)=O)CC2)c2nc(N)ncc2c1 [10507]
#   5. CCc1cc(O)ccc1-c1ccc2c(-c3ncc[nH]3)[nH]nc2c1 [10565]
#   6. CN(c1ncnc2[nH]ccc12)C1CCC(CS(=O)(=O)N2CCCC(OP(=O)(O)O)C2)CC1	[12237]
#   7. CN1CCN(Cc2ccc(NC(=O)c3ccc(Cl)c(C#Cc4cncc(N5CCOCC5)c4)c3)cc2C(F)(F)F)CC1 [13036]
#   8. CNC(=O)n1ccc2cc(Oc3ccnc(NC(=O)c4ccc(C5CCN(CCO)CC5)cc4)c3)c(OCCO)cc21 [14359]
#   9. COc1cc(OC)c(Cl)c(N2Cc3cnc4[nH]ccc4c3N(C)C2=O)c1F [18095]
#   10. COc1cc(OC)c(F)c(COc2cnc(Nc3cnn(CCO)c3)nc2)c1F [18128]
#   11. COc1cc(OC)c(F)c(N2Cc3cnc4[nH]ccc4c3N(C)C2=O)c1F	[18138]
#   12. COc1cc(OC)cc(-c2nn(CCCCCN3CCN(C(=O)C=CC(F)(F)F)CC3)c3ncnc(N)c23)c1 [18184]
#   13. COc1cc2ncc(-c3cccc(NC4CCNC4)n3)n2cc1-c1cnn(C)c1	[19874]
#   14. Cc1[nH]nc2ccc(-c3cc(NC(CO)c4ccccc4)cnc3-c3ccc(F)cc3O)cc12 [23761]
#   15. Cc1cc(-c2cc(O)ccc2Cl)cc2nnc(Nc3ccc(OCCN4CCCC4)cc3)nc12 [24266]
#   16. Cc1cc2cnc3[nH]ccc3c2n1C1CN(C(=O)CC#N)CCC1C [25335]
# 2+ (235)
# 1+ (2477)

In [ ]:
# CUTOFF = pChEMBL 8.5
# 8+ (1)
#   1. CNC(=O)c1ncc(C#Cc2cc(C(=O)Nc3ccc(CN4CCN(CCO)CC4)c(C(F)(F)F)c3)ccc2C)n1C [14272]
# 7+ (2)
#   2. Cc1cc(-c2cc(O)ccc2Cl)cc2nnc(Nc3ccc(OCCN4CCCC4)cc3)nc12 [24266]
# 6+ (2)
# 5+ (5)
#   3. CCN(CC)CCNC(=O)c1c(C)[nH]c2c1=CCCC=2c1c(O)[nH]c2ccc(F)cc12 [8734]
#   4. CNC1CC2OC(C1OC)n1c3ccccc3c3c4c(c5c6ccccc6n2c5c31)C(=O)NC4 [14404]
#   5. OCC(Nc1cnc(-c2cc(Cl)ccc2O)c(-c2ccc3cnccc3c2)c1)c1ccccc1 [36186]
# 3+ (97)
# 1+ (5412)

In [ ]:
# CUTOFF = pChEMBL 7.5
# 12+ (1)
#   1. CNC(=O)c1ncc(C#Cc2cc(C(=O)Nc3ccc(CN4CCN(CCO)CC4)c(C(F)(F)F)c3)ccc2C)n1C [14272]
# 10+ (3)
#   2. Cc1[nH]nc2ccc(-c3cc(NC(CO)c4ccccc4)cnc3-c3ccc(F)cc3O)cc12 [23761]
#   3. OCC(Nc1cnc(-c2cc(Cl)ccc2O)c(-c2ccc3cnccc3c2)c1)c1ccccc1	[36186]
# 8+ (12)
#   4. CCN1CCN(c2ccc(Nc3ncc4scc(-c5cccc(NS(C)(=O)=O)c5)c4n3)cc2)CC1	[9163]
#   5. CN1CCN(Cc2ccc(NC(=O)c3ccc(Cl)c(C#Cc4cncc(N5CCOCC5)c4)c3)cc2C(F)(F)F)CC1 [13036]
#   6. COC(=O)c1ccc2c(C(Nc3ccc(N(C)C(=O)CN4CCN(C)CC4)cc3)c3ccccc3)c(O)[nH]c2c1 [15074]
#   7. Cc1[nH]nc2ccc(-c3cc(NC(CO)c4ccccc4)cnc3-c3cc(Cl)ccc3O)cc12 [23757]
#   8. Cc1cc(-c2cc(O)ccc2Cl)cc2nnc(Nc3ccc(OCCN4CCCC4)cc3)nc12 [24266]
#   9. Cc1ccc(NC(=O)c2ccc(C)c(C(F)(F)F)c2)cc1C#Cc1nn(C2CCC(O)CC2)c2ncnc(N)c12 [26307]
#   10. Cn1cc(-c2cnc3c(-c4csc(C(=O)NC5CCCCC5N)c4)cnn3c2)cn1	[28975]
#   11. Nc1nc(Nc2ccc3c(c2)CCC(N2CCCC2)CC3)nn1-c1cc2c(nn1)-c1ccccc1CCC2 [32349]
#   12. O=C1Nc2ccccc2C1=Cc1c[nH]nc1-c1ccccc1[N+](=O)[O-] [35580]
# 3+ (893)
# 1+ (16125)

In [ ]:
# CUTOFF = pChEMBL 6.5
# 17+ (1)
#   1. CCn1cc(-c2ccc3c(c2)CCN3C(=O)Cc2cccc(OC(F)(F)F)c2)c2c(N)ncnc21 [10964]
# 15+ (3)
#   2. CNC(=O)c1ncc(C#Cc2cc(C(=O)Nc3ccc(CN4CCN(CCO)CC4)c(C(F)(F)F)c3)ccc2C)n1C [14272]
#   3. Cn1cc(-c2cnc3c(-c4csc(C(=O)NC5CCCCC5N)c4)cnn3c2)cn1 [28975]
# 13+ (7)
#   4. CC(C)Cn1c2ccc(Nc3ncccn3)cc2c2c3c(c4c(c21)CCc1nn(C)cc1-4)C(=O)NC3 [5165]
#   5. COC(=O)c1ccc2c(C(=Nc3ccc(N(C)C(=O)CN4CCN(C)CC4)cc3)c3ccccc3)c(O)[nH]c2c1	[15040]
#   6. COC(=O)c1ccc2c(C(Nc3ccc(N(C)C(=O)CN4CCN(C)CC4)cc3)c3ccccc3)c(O)[nH]c2c1 [15074]
#   7. Cc1ccc(NC(=O)c2ccc(C)c(C(F)(F)F)c2)cc1C#Cc1nn(C2CCC(O)CC2)c2ncnc(N)c12 [26307]
# 8+ (45)
# 3+ (2313)
# 1+ (26593)

In [ ]:
# NO CUTOFF
# 23+ (2)
#   1. Nc1ncnc2c1c(-c1cccc(O)c1)cn2C1CC(CO)C1 [32934]
#   2. Nc1ncnc2c1c(-c1cccc(O)c1)cn2C1CCCC1 [32936]
# 18+ (14)
#   3. CC(C)(C)c1cc(NC(=O)Nc2ccc(Nc3ncnc4c3OCCCN4)cc2)no1 [4731]
#   4. CC1(C)C(=O)N(C2CCc3c(O)cccc32)c2nc(Nc3ccccc3)ncc21	[6807]
#   5. CC1(C)OCC(=O)Nc2cc(Nc3nc(NCC(F)(F)F)c4occc4n3)ccc21 [6929]
#   6. CCn1cc(-c2ccc3c(c2)CCN3C(=O)Cc2cccc(OC(F)(F)F)c2)c2c(N)ncnc21 [10964]
#   7. CNc1ncnc(-c2cccnc2Oc2ccc(F)c(C(=O)Nc3cc(C(F)(F)F)ccc3N(C)CCCN(C)C)c2)n1 [14725]
#   8. COC(=O)Nc1ccc(-c2c(OC3CCN(C)CC3)nc3c(-c4ccc(OC)c(OC)c4)cnn3c2N)cc1	[14876]
#   9. COC(=O)c1ccc2c(C(=Nc3ccc(N(C)C(=O)CN4CCN(C)CC4)cc3)c3ccccc3)c(O)[nH]c2c1 [15040]
#   10. COC1C(N(C)C(=O)c2ccccc2)CC2OC1(C)n1c3ccccc3c3c4c(c5c6ccccc6n2c5c31)C(=O)NC4 [15192]
#   11. Cc1cc(Nc2ncc(Cl)c(Nc3ccccc3S(=O)(=O)C(C)C)n2)c(OC(C)C)cc1C1CCNCC1 [25085]
#   12. Cn1cc(-c2cnc3c(-c4csc(C(=O)NC5CCCCC5N)c4)cnn3c2)cn1 [28975]
#   13. Cn1cc(-c2cnc3c(-c4csc(C(=O)NCC(F)(F)F)c4)cnn3c2)cn1 [28976]
#   14. O=C(NCC(F)(F)F)c1cc(-c2cnn3cc(-c4ccc(OCCN5CCCCC5)cc4)cnc23)cs1 [33936]
# 13+ (32)
# 8+ (128)
# 3+ (4303)
# 2+ (10016)
# 1+ (37004, all)